In [1]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "master.hadoop.akcheck.ru:9092") \
  .option("subscribe", "test") \
  .load()


In [5]:
df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [6]:
dstream = df.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")

In [7]:
dstream

DataFrame[topic: string, key: string, value: string]

In [19]:
from pyspark.sql.functions import from_json, col

In [20]:
from pyspark.sql.types import *

In [10]:
schema = StructType([
    StructField("method", StringType()),
])

In [11]:
from pyspark.sql.functions import lit

In [13]:
dstream = dstream.withColumn('json', from_json(col('value'), schema))

In [14]:
dstream

DataFrame[topic: string, key: string, value: string, json: struct<method:string>]

In [15]:
dstream = dstream.withColumn('topic', lit('student00-frontend'))

In [22]:
dstream

DataFrame[topic: string, key: string, value: string, json: struct<method:string>]

In [23]:
dstream = dstream.withColumn('method', col('json.method').alias('method'))

In [24]:
dstream

DataFrame[topic: string, key: string, value: string, json: struct<method:string>, method: string]

In [25]:
query = dstream.writeStream.format("console") \
            .option("truncate", "false") \
            .outputMode("append") \
            .option("checkpointLocation", "checkpoints") \
            .start() \
            .awaitTermination()

KeyboardInterrupt: 

Получаем такой формат для Frontend-a!

In [6]:
log_time = {
    "time":"2023-09-30T00:32:45.306747001+03:00",
    "stream":"stdout",
    "logtag":"F",
    "kubernetes":
        {
            "pod_name":"frontend-7bd4f5dcdd-9zq5r",
            "namespace_name":"akhcheck",
            "pod_id":"ed431839-2d0f-43ec-bdf8-cd3feae307a1",
            "labels":
                {"app":"frontend","pod-template-hash":"7bd4f5dcdd"},
            "host":"lobachevsky","container_name":"myapp",
            "docker_id":"7470b8081364efd186636088eddce03be944ba284b5a4b1a4854f9e47e8bc06c",
            "container_hash":"deployregistry.akhcheck.ru/frontend@sha256:f37304aa6bfe150cf93935956ac3f0c100fd699150bf2aaaac12e23a2862c6b4",
            "container_image":"deployregistry.akhcheck.ru/frontend:185"
        },
    "remote":"10.42.0.78",
    "host":"-","user":"-","method":"GET","path":"/course_registration","code":"200","size":"10417","referer":"-","agent":"Mozilla/5.0 (iPhone; CPU iPhone OS 16_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1"
}

Создаем топик для Frontend-a!

```shell
kafka-topics.sh --create --topic student00-frontend --partitions 5 --bootstrap-server 10.128.0.34:9092
```

In [17]:

ds = dstream \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.34:9092") \
  .option("checkpointLocation", "checkpoints-frontend") \
  .start().awaitTermination()

KeyboardInterrupt: 